### Step 0: specify the Free-form API urls


In [1]:
#Free form SDS API url
url='https://synthetic-data-generator-oaqw25.ai-workbench.eng-ml-l.vnu8-sqze.cloudera.site/synthesis/freeform' 
#Free form eval SDS API url
url_eval = 'https://synthetic-data-generator-oaqw25.ai-workbench.eng-ml-l.vnu8-sqze.cloudera.site/synthesis/evaluate_freeform'


### Step 1: Real Data Reference
**Purpose**: Load sample real-world lending data to understand structure for synthetic generation  
**Key Features**:  
- Contains 27 fields including loan amount, interest rate, employment details, and credit history  
- Example fields: `loan_amnt` (loan amount), `int_rate` (interest rate), `emp_title` (job title)  
- Used as reference for generating realistic synthetic data patterns


In [2]:
#These are the lending datasets and how they look like
import pandas as pd
OutputFile='lending_club_loan_two.csv'
data = pd.read_csv(OutputFile)
data

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396025,10000.0,60 months,10.99,217.38,B,B4,licensed bankere,2 years,RENT,40000.0,...,6.0,0.0,1990.0,34.3,23.0,w,INDIVIDUAL,0.0,0.0,"12951 Williams Crossing\r\nJohnnyville, DC 30723"
396026,21000.0,36 months,12.29,700.42,C,C1,Agent,5 years,MORTGAGE,110000.0,...,6.0,0.0,43263.0,95.7,8.0,f,INDIVIDUAL,1.0,0.0,"0114 Fowler Field Suite 028\r\nRachelborough, ..."
396027,5000.0,36 months,9.99,161.32,B,B1,City Carrier,10+ years,RENT,56500.0,...,15.0,0.0,32704.0,66.9,23.0,f,INDIVIDUAL,0.0,0.0,"953 Matthew Points Suite 414\r\nReedfort, NY 7..."
396028,21000.0,60 months,15.31,503.02,C,C2,"Gracon Services, Inc",10+ years,MORTGAGE,64000.0,...,9.0,0.0,15704.0,53.8,20.0,f,INDIVIDUAL,5.0,0.0,"7843 Blake Freeway Apt. 229\r\nNew Michael, FL..."


### Step 2a: Synthetic Data Generation
**Key Parameters**:
- **Model**: Meta Llama-3.1-70B (large language model running on CAII)  
- **Temperature 1.0**: High creativity for diverse outputs
- **max_tokens 8192**: Enough tokens to generate each output fully.
- **Examples**: Uses `ExamplesLoanData.json` for pattern learning
  -  Fields need to match fields in the prompt in the same sequence.
  -  Reduct PII
  -  Ensure diversity of examples
  -  Consistency in categories: Ensure fields like `home_ownership` use predefined values (RENT, OWN, MORTGAGE, OTHER).  
- **Prompt Structure**:  
  - Specifies all 27 fields with precise definitions  
  - Requires logical relationships between variables (e.g., interest rates matching credit risk)
  - Explain each field clearly in the expected sequence.


In [3]:
%%time
import requests
import os

# Get API key from environment variable if within CDSW app/session
api_key = os.environ.get('CDSW_APIV2_KEY')


# URL for synthesis


# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# If API key exists, add it to the headers
if api_key:
    headers['Authorization'] = f'Bearer {api_key}'
else:
    print("Warning: No API key provided")

# Payload for data synthesis
payload = {
  #Use CAII models
  "inference_type": "CAII",
  "caii_endpoint": "https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions",
  "model_id": "meta/llama-3.1-70b-instruct",
  #Use AWS Bedrock models
  #"inference_type": "aws_bedrock",
  #"model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",

  "is_demo": False,
  "num_questions": 200,
  "custom_prompt": """
 You need to create profile data for the LendingClub company which specialises in lending various types of loans to urban customers.
 

You need to generate the data in the same order for the following  fields (description of each field is followed after the colon):

loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term: The number of payments on the loan. Values are in months and can be either 36 months or 60 months.
int_rate: Interest Rate on the loan
installment: The monthly payment owed by the borrower if the loan originates.
grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e A5)
emp_title: The job title supplied by the Borrower when applying for the loan.
emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc: The self-reported annual income provided by the borrower during registration.
verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
issue_d: The month which the loan was funded
loan_status: Current status of the loan
purpose: A category provided by the borrower for the loan request.
title: The loan title provided by the borrower
dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
earliest_cr_line: The month the borrower's earliest reported credit line was opened
open_acc: The number of open credit lines in the borrower's credit file.
pub_rec: Number of derogatory public records
revol_bal: Total credit revolving balance
revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc: The total number of credit lines currently in the borrower's credit file
initial_list_status: The initial listing status of the loan. Possible values are – W, F
application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
mort_acc: Number of mortgage accounts.
pub_rec_bankruptcies: Number of public record bankruptcies
address: The physical address of the person

Ensure PII from examples such as addresses are not used in the generated data to minimize any privacy concerns.
""",
  "model_params": {
    "temperature": 1.0, # range 0-2 tyically, low temperature gives high accuracy, high temperature gives diversity
    "top_p": 1.0,
    "top_k": 250,       
    "max_tokens": 8192
  },
  "use_case": "custom",
  "topics": [
    "Financial data"
  ],
  "example_path": "ExamplesLoanData.json"
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Display the response
print(response.status_code)
print(response.json())


200
{'job_name': 'synth_job_3bfa', 'job_id': 'p4n8-0osu-ocpg-9wlz'}
CPU times: user 110 ms, sys: 3.53 ms, total: 114 ms
Wall time: 2.27 s


### Step 2b: Filter data based on expected columns
**Filtering**:  
- Check if all samples have the exact columns as expected
- Remove those with different column names than expected


In [25]:
import pandas as pd
import json
InputFile='freeform_data_llama_20250416T063557618_final.json'

ExpectedKeys=['Seeds', 'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'title', 'dti', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'application_type', 'mort_acc', 'pub_rec_bankruptcies', 'address']

with open(InputFile,'r') as f:
  data_j = json.load(f)
FilteredData=[]
for i in data_j:
    if (list(i.keys())==ExpectedKeys)==True:
      FilteredData.append(i)

InputFile='Filtered_' + InputFile
with open(InputFile,'w') as f:
  json.dump(FilteredData,f)


### Step 2c: Synthetic Data Inspection
**Sample Output Analysis**:  
- Generated all 27 fields as the original data
- Co-dependent variables vary as expected (i.e. a lower loan grade pays higher interest on the loan)
- Contains plausible occupations (`Software Engineer`, `Teacher`)  
- Addresses follow realistic patterns (`2345 Tech Lane San Jose, CA 95123`)  


In [26]:
df=pd.DataFrame(FilteredData)
df

,Seeds,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,Financial data,32000.0,60 months,13.99,675.42,B,B2,Senior Accountant,6 years,RENT,...,14.0,0.0,14356.0,42.1,23.0,w,INDIVIDUAL,0.0,0.0,"666 Jennifer Way\r\nCarlosville, AL 76543"
1,Financial data,14250.0,36 months,10.94,452.07,B,B5,Physical Therapist,3 years,MORTGAGE,...,15.0,0.0,9218.0,50.4,28.0,f,INDIVIDUAL,1.0,0.0,"253 Grant Street\r\nFowlerville, MI 49221"
2,Financial data,10000.0,36 months,12.46,319.49,C,C3,Business Analyst,7 years,RENT,...,12.0,0.0,17488.0,46.2,24.0,w,INDIVIDUAL,0.0,0.0,"8701 Jennifer Plaza\r\nDennisside, NC 45828"
3,Financial data,25000.0,60 months,15.49,562.44,C,C5,Electrician,8 years,MORTGAGE,...,16.0,0.0,30975.0,61.2,37.0,f,INDIVIDUAL,2.0,0.0,"5314 Oakwood Street\r\nRennerberg, CA 90291"
4,Financial data,39000.0,60 months,16.23,819.33,D,D4,Database Administrator,10+ years,RENT,...,19.0,0.0,41742.0,55.6,41.0,w,INDIVIDUAL,0.0,0.0,"1434 Terry Crossing\r\nDuaneberg, ID 04783"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Financial data,5000.0,36 months,15.95,165.47,E,E1,Computer Systems Analyst,3 years,RENT,...,7.0,0.0,4119.0,23.1,14.0,w,INDIVIDUAL,0.0,0.0,"85082 Cordova Land Apt. 604\r\nPort Nicholas, ..."
194,Financial data,11200.0,60 months,14.41,251.45,C,C3,Registered Nurse,5 years,MORTGAGE,...,11.0,0.0,19045.0,50.2,24.0,f,INDIVIDUAL,2.0,0.0,"25630 Danielle Place Apt. 377\r\nWest Kyle, DE..."
195,Financial data,18500.0,36 months,12.99,585.17,B,B2,Business Development Manager,10+ years,RENT,...,14.0,0.0,25669.0,54.3,29.0,w,INDIVIDUAL,0.0,0.0,"90434 Price Station Apt. 897\r\nEast Kevin, SD..."
196,Financial data,16875.0,60 months,13.71,362.26,C,C1,Science Teacher,7 years,MORTGAGE,...,13.0,0.0,22146.0,63.9,31.0,f,INDIVIDUAL,2.0,0.0,42782 Pearson Overpass Apt. 172\r\nPort Adelin...


### Section 3a: LLM-Based Evaluation
**Evaluation Process**:  
1. **Prompt**:
   - Provide all required definitions and checks needed for evaluation.
   - Ensure consistency of definitions with the generation prompt
   - Explain clearly how the LLM will score
   - give the LLM examples on what to check. For example, asking the LLM specifically to check for data co-dependencies, temporal consistency and field validity improves evaluation quality.

2. **Scoring**:
   - Specify a scale (1-5 scale in this example):
   - Penalties for mistakes (e.g. field name mismatches, inconsistent field relationships, etc)


3. **Parameter tuning**: Use low `temperature` (e.g., 0.0) to prioritize accuracy over diversity.  


In [27]:
import requests
import os
#********************Accessing Application**************************
# Get API key from environment variable if withinin CDSW app/session.
# To get your API key for using outside CDSW app/session follow given link.
# https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-api-v2.html
api_key = os.environ.get('CDSW_APIV2_KEY')


# Below is your application API URL, you can look at swagger documentation for all existing # endpoints for current application
# https://<application-subdomain>.<workbench-domain>/docs--> will take user to swagger documentaion
# Link to application can be found on application details page within CAI Workbench.


# URL for evaluation

# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'  # Format as specified in the documentation
}   

# The prompt for evaluation
custom_prompt = """Below is financial data synthesized by an LLM which have the following fields (description of each field is followed after the colon):

loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term: The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate: Interest Rate on the loan
installment: The monthly payment owed by the borrower if the loan originates.
grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e A5)
emp_title: The job title supplied by the Borrower when applying for the loan.
emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc: The self-reported annual income provided by the borrower during registration.
verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
issue_d: The month which the loan was funded
loan_status: Current status of the loan
purpose: A category provided by the borrower for the loan request.
title: The loan title provided by the borrower
dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
earliest_cr_line: The month the borrower's earliest reported credit line was opened
open_acc: The number of open credit lines in the borrower's credit file.
pub_rec: Number of derogatory public records
revol_bal: Total credit revolving balance
revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc: The total number of credit lines currently in the borrower's credit file
initial_list_status: The initial listing status of the loan. Possible values are – W, F
application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
mort_acc: Number of mortgage accounts.
pub_rec_bankruptcies: Number of public record bankruptcies
address: The physical address of the person:

Rate the quality of the generated whether they reflect real financial data and the variables make sense when considered to together.

For example, int_rate which represents interest rate should reflect real world interest rates during the issue_d period (The month which the loan was funded) but also reflect the risk profile of the person, dti, earliest_cr_line and other fields.
For example, the installment monthly amount need computed from the loan amount, the term and the int_rate and be consistent.

Make sure you analyze as many condependence between the fields and subtract 1 point for each co-dependency that does not reflect real data.
Subtract 2 points if there is a mistake in the field keys or field values.
Subtract 2 points for each other mistake you see with the data.
Give a score rating 1-5 for the given data. 
"""




# Model parameters
model_params = {
    "temperature": 0.0,
    "top_p": 1.0,
    "top_k": 250,
    "max_tokens": 4096
}

payload = {
    "export_type": "local",
    "display_name": "LendingData",
    "import_path": InputFile,
    "import_type": "local",
    #Use CAII models
    "inference_type": "CAII",
    "caii_endpoint": "https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions",
    "model_id": "meta/llama-3.1-70b-instruct",
        
    #Use AWS Bedrock models
    #"inference_type": "aws_bedrock",
    #"model_id": "us.anthropic.claude-3-5-sonnet-20241022-v2:0",

    "use_case": "custom",
    "is_demo": False,
    "custom_prompt": custom_prompt,
    "model_params": model_params
}

responseEval = requests.post(url_eval, headers=headers, json=payload)

# Print the response
print(responseEval.status_code)
print(responseEval.json())


200
{'job_name': 'LendingData_e03e', 'job_id': 'z1rz-6ha7-jvm6-4fab'}


## Step 3b: Example LLM-as-a-judge output
- Shows the sample in question.  
- Provides a score and a justification for the score which can be used for further filtering.  


In [34]:
import pandas as pd
import json
##############
#Replace this with the output of the LLM-as-a-judge step
LLMJUDGEOUT='row_data_llama_20250416T074946051_evaluated.json'
###############
with open(LLMJUDGEOUT,'r') as f:
  data_j = json.load(f)
print(json.dumps(data_j,indent=4))

{
    "average_score": 4.0,
    "min_score": 4,
    "max_score": 4,
    "evaluated_rows": [
        {
            "row": {
                "Seeds": "Financial data",
                "loan_amnt": 32000.0,
                "term": "60 months",
                "int_rate": 13.99,
                "installment": 675.42,
                "grade": "B",
                "sub_grade": "B2",
                "emp_title": "Senior Accountant",
                "emp_length": "6 years",
                "home_ownership": "RENT",
                "annual_inc": 82000.0,
                "verification_status": "Not Verified",
                "issue_d": "Jun-2016",
                "loan_status": "Fully Paid",
                "purpose": "debt_consolidation",
                "title": "Debt Consolidation",
                "dti": 20.54,
                "earliest_cr_line": "Apr-2001",
                "open_acc": 14.0,
                "pub_rec": 0.0,
                "revol_bal": 14356.0,
                "revol_util": 4

## Step 3c: Filtering and Conversion to Tabular Data
- Filter low-quality rows and convert valid data into a DataFrame.  
- Choose a score threshold (e.g., >3.5) to balance quality and quantity based on business needs.  


In [35]:
AllRows=[]
for i in data_j['evaluated_rows']:
  del i['row']['Seeds']
  if i['evaluation']['score'] > 3.5:
      AllRows.append(i['row'])

import pandas as pd
df=pd.DataFrame.from_records(AllRows)
df


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,32000.0,60 months,13.99,675.42,B,B2,Senior Accountant,6 years,RENT,82000.0,...,14.0,0.0,14356.0,42.1,23.0,w,INDIVIDUAL,0.0,0.0,"666 Jennifer Way\r\nCarlosville, AL 76543"
1,25000.0,60 months,15.49,562.44,C,C5,Electrician,8 years,MORTGAGE,75000.0,...,16.0,0.0,30975.0,61.2,37.0,f,INDIVIDUAL,2.0,0.0,"5314 Oakwood Street\r\nRennerberg, CA 90291"
2,14250.0,36 months,10.94,452.07,B,B5,Physical Therapist,3 years,MORTGAGE,64000.0,...,15.0,0.0,9218.0,50.4,28.0,f,INDIVIDUAL,1.0,0.0,"253 Grant Street\r\nFowlerville, MI 49221"
3,10000.0,36 months,12.46,319.49,C,C3,Business Analyst,7 years,RENT,56000.0,...,12.0,0.0,17488.0,46.2,24.0,w,INDIVIDUAL,0.0,0.0,"8701 Jennifer Plaza\r\nDennisside, NC 45828"
4,14250.0,60 months,13.49,274.78,D,D3,Clinical Research Coordinator,5 years,RENT,60000.0,...,14.0,0.0,16750.0,30.5,32.0,f,JOINT,0.0,0.0,"7228 Crane Hwy Apt. 585\r\nNorth Jason, MT 23025"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,5000.0,36 months,15.95,165.47,E,E1,Computer Systems Analyst,3 years,RENT,42000.0,...,7.0,0.0,4119.0,23.1,14.0,w,INDIVIDUAL,0.0,0.0,"85082 Cordova Land Apt. 604\r\nPort Nicholas, ..."
194,11200.0,60 months,14.41,251.45,C,C3,Registered Nurse,5 years,MORTGAGE,60000.0,...,11.0,0.0,19045.0,50.2,24.0,f,INDIVIDUAL,2.0,0.0,"25630 Danielle Place Apt. 377\r\nWest Kyle, DE..."
195,18500.0,36 months,12.99,585.17,B,B2,Business Development Manager,10+ years,RENT,105000.0,...,14.0,0.0,25669.0,54.3,29.0,w,INDIVIDUAL,0.0,0.0,"90434 Price Station Apt. 897\r\nEast Kevin, SD..."
196,16875.0,60 months,13.71,362.26,C,C1,Science Teacher,7 years,MORTGAGE,70000.0,...,13.0,0.0,22146.0,63.9,31.0,f,INDIVIDUAL,2.0,0.0,42782 Pearson Overpass Apt. 172\r\nPort Adelin...


## Step 4: Saving to CSV
- **Export**: Save the cleaned data to a CSV file for downstream use.  
- **Format consistency**: Use a delimiter like `\t` to avoid conflicts with existing data (e.g., commas in addresses).  


In [36]:
OutputFile='data_tab_separated.csv'
df.to_csv(OutputFile, sep='\t')
